In [ ]:
!gdown 1seeIzLeGu8MZyCTnJx2nQpilKBHIcHvK #crs.txt
!gdown 1iHPYor7CKiJY95Attt2nEFtY8rrQD0oU #data.zip
!gdown 1gbU9CaSHfsDhALcGZd3hcbSpi9icPiLw #gaziera.zip
!gdown 1kPgjP5qSqseDZtfGseWuhiAc21BWm7pK #qdarif under sampled

!gdown 1zRKg1MWjhGkeSzJNGFhhaUIIO-FljulR
!gdown 15Bpveev30iV46Ehlk5pQN5hHqqmW_Hdu
!gdown 1ZNYPklSzjYnD1fYtd9eYu0OIcWJmK6cK
!unzip data.zip
!pip install -q geopandas
!pip install -q contextily
!pip install -q mapclassify
!pip install -q streamlit-folium
!pip install -q folium
!pip install -q rioxarray
!mkdir curated_data
!mv gaziera_83_columns.csv curated_data/
!mv managil_83_columns.csv curated_data/
!mv qadarif_83_columns.csv curated_data/
!mv qadarif_83_columns_under_sampled.csv curated_data/

In [ ]:
#some useful libraries and functions

import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, cross_validate
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import Polygon, Point
from pyproj import CRS
from shapely import wkt
from tqdm import tqdm
from matplotlib.colors import ListedColormap
from sklearn import tree
import  sklearn.feature_selection
from sklearn import feature_selection as fs
from sklearn import preprocessing as ps
from sklearn import decomposition as dc
import joblib
import numpy as np
from sklearn import ensemble as ens
from sklearn.pipeline import Pipeline


def convert_dataframe_to_geodataframe(df, crs):
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry=df.geometry, crs=crs)
    return gdf


In [ ]:
import rioxarray as rx
import xarray as xr
import math
from sklearn.model_selection import GridSearchCV

# The pipeline class which is the main output from the training pipline,
# The class has three main functions and an initiation function

class PipelineW():
    # The init function takes a scikit learn pipeline layers array and a list of crops (anythign else is converted to other)
    def __init__(self, pipeline, cropnames=['Sorghum'], train=False):
        self.train=train
        self.pipeline=Pipeline(pipeline)
        self.cropnames=cropnames
        self.temp=None
        self.crs=crs

    #The gen function takes the labels gdf from the downlowder -> gets the images downloaded -> and does some feature engineering
    # it then outputs the gdf in the format ready to be processes by the fit and predict functions.

    def gen(self, labelsgdf=None, train=False):
        self.train=train
        gdf=self.readgdf(labelsgdf)
        return gdf

    #Takes in the output form the gen function (train=true) and trains the pieplien model on it
    #basically the same as the scikit learn fit functin of the pipeline class
    def fit(self, gdf):
        Y=gdf.Crop_Type.values
        X=gdf.drop(['State', 'fieldID', 'Rainfed', 'Crop_Type', 'Year', 'y_bins', 'x_bins'], axis=1, errors='ignore')
        self.pipeline.fit(X, y=Y)
        return

    #Takes in the output form the gen function (train=false) and predicts the class
    #basically the same as the scikit learn predict functin of the pipeline class
    def predict(self, X):
        predy=self.pipeline.predict(X)
        return predy

    #Takes in the output form the gen function (train=false) and outputs the best model
    #basically the same as the scikit learn predict functin of the pipeline class

    def selectbest(self,gdf, param_grid, pipe=False):
        Y=gdf.Crop_Type.values
        X=gdf.drop(['State', 'fieldID', 'Rainfed', 'Crop_Type', 'Year', 'y_bins', 'x_bins'], axis=1, errors='ignore')
        if not pipe:
            pipe=self.pipeline
        search = GridSearchCV(pipe, param_grid, n_jobs=4, verbose=4, scoring=['precision', 'recall','f1_micro'], refit='f1_micro')
        search.fit(X, Y)
        print("Best parameter (CV score=%0.3f):" % search.best_score_)
        print(search.best_params_)
        self.pipeline=search.best_estimator_
        return search

    #Function that reads the images tiff files and formats them in required geogson format
        #[geometry, Crop_Type, Band_Date_pairs ...etc]
    def readgdf(self, labelsgdf):
        print('Reading tiff files')
        gdf=pd.DataFrame()
        labelsgdf=labelsgdf.sample(frac=1)
        gdfL=[]
        gdfl=[]
        for i, field in tqdm(labelsgdf.iterrows()):
            L=[]

            for imgPath in field['imgPaths']:

                nxr= rx.open_rasterio(imgPath)#, parse_coordinates=crs)
                L.append(nxr[:14,:,:])
            nxr=xr.concat(L, "T")
            ###group data
            nxr=self.group_xarr_max(nxr)
            ##################
            nxr.name='data'
            nxr=nxr.to_dataframe()
            nxr.drop('spatial_ref', axis=1,  inplace=True)
            nxr=nxr.unstack(level=['band', 'T'])
            nxr.reset_index(inplace=True)
            rows=nxr.shape[0]
            if self.train:
                nxr['Crop_Type']=[self.replace_label(field['Crop_Type'])]*rows
            nxr['State']=[field['State']]*rows
            nxr['Year']=[field['Year']]*rows
            nxr['Rainfed']=[field['Rainfed']]*rows
            nxr['fieldID']=[field['fieldID']]*rows
            if i==0:
                gdf=nxr
                continue
            gdfl.append(nxr)
            if i%200==0 or i == (len(labelsgdf)-1):
                gdfL.append(pd.concat(gdfl, axis=0))
                gdfl=[]
        gdf=pd.concat(gdfL, axis=0)

#         geo=gpd.points_from_xy(gdf.x, gdf.y, crs=crs)
#         gdf=gpd.GeoDataFrame(gdf, geometry=geo)
#         self.temp=gdf
        print(gdf.shape)
        return gdf

    def replace_label(self,crop_name):
        if crop_name is None:
            crop_name=0
        elif crop_name in self.cropnames:
            crop_name=self.cropnames.index(crop_name)+1
        else:
            crop_name=0
        return crop_name

    def group_xarr(self, nxr):
        xlen=nxr.shape[3]
        binsx=math.ceil(xlen/3)
        ylen=nxr.shape[2]
        binsy=math.ceil(ylen/3)

        nxrg=nxr.groupby_bins('x',binsx).median()
        nxrg=nxrg.groupby_bins('y',binsy).median()

        return nxrg

    def group_xarr_max(self, nxr):
        xlen=nxr.shape[3]
        binsx=math.ceil(xlen/5)
        ylen=nxr.shape[2]
        binsy=math.ceil(ylen/5)

        nxrg=nxr.groupby_bins('x',binsx).max()
        nxrg=nxrg.groupby_bins('y',binsy).max()

        return nxrg


# class group_transformer(BaseEstimator, TransformerMixin):
#     def __init__(self, fac=(3, 3)):
#         self.gdf=None
#         self.fac=fac
#         self.gdf=gpd.GeoDataFrame()

#     def fit (self, X, y=None):
#         return

#     def transform(self, X, y=None):
#         self.gdf['cluster']=list(zip(round(all_gdf.geometry.x*fac[0], fac[1])/fac[0],round(all_gdf.geometry.y*fac[0],fac[1])/fac[0]))
#         self.gdf=gpd.GeoDataFrame(self.gdf.groupby(by=['cluster','state', 'Crop_Type'], as_index=False).median(), geometry=self.gdf['geometry'])


<h1>Binary Sorghum DT NDVI PCA**<h1>

In [ ]:
%%time
with open('crs.txt', 'r') as file:
    crs = CRS.from_wkt(file.read())
# all_gdf=convert_dataframe_to_geodataframe(pd.read_csv("./curated_data/new_data.csv"), crs).drop('Unnamed: 0', axis=1)


In [ ]:
#load labels gdf passed from downloader
labelsgdf=joblib.load('labels.joblib')

In [ ]:
#Setting pipeline layers
norm1=ps.MinMaxScaler()
kb1=fs.SelectKBest(score_func=fs.chi2, k=30)
cl1=ens.RandomForestClassifier(n_estimators=10, min_samples_leaf=2000) #max_depth = 3

#initiating PieplineW class, by passing it pipeline layers array and cropnames
model1 = PipelineW([('minmax', norm1),('f_class_best', kb1), ('RF', cl1)], cropnames=['Sorghum'])

#generates training geogson
gdf=model1.gen(labelsgdf, train=True)

In [ ]:
#stores or loads featured gegoson files

joblib.dump(gdf, 'gdfGrouped5Sorghum.joblib')
#gdf=joblib.load('gdfGroupedCotton.joblib')

In [ ]:
#fitting pipeline

model1.fit(gdf)

In [ ]:
#Features selection set up

#Setting pipeline layers
norm1=ps.MinMaxScaler()
kb1=fs.SelectKBest(score_func=fs.chi2) #score_func=fs.f_classif, k=30
cl1=ens.RandomForestClassifier(n_estimators=10) #max_depth = 3 #n_estimators=10, min_samples_leaf=2000
model1 = PipelineW([('minmax', norm1),('f_class_best', kb1), ('RF', cl1)], cropnames=['Cotton'])

#settign parametere to search
param_grid = {
    #"f_class_best__score_func": [fs.f_classif, fs.chi2],
    "f_class_best__k":[20,30,40],
    #"RF__max_depth": [3,5,7,9,11],
    #"RF__n_estimators":[10,15,20],
    "RF__min_samples_leaf":[1000,2000,3000],
    "RF__criterion":["gini", "entropy", "log_loss"],
}


#finding the best fitting model
model1.selectbest(gdf,param_grid)

In [ ]:
#predicting classes
preds=model1.predict(X)

In [ ]:
#testing model
print(metrics.classification_report(Y, preds, zero_division=0))
cm=metrics.confusion_matrix(Y, preds, labels=(1, 0))
print(cm)
metrics.ConfusionMatrixDisplay(cm).plot()

In [ ]:
#storing or loading model

joblib.dump(model1, './groupRFoptCotton.joblib')
#model1=joblib.load('./groupRFoptCotton.joblib')